In [1]:
rdd = sc.textFile("../data/taxi-data-sorted-small.csv.bz2")
rdd = rdd.map(lambda x: x.split(','))

In [2]:
# import bz2
# with bz2.open("../data/taxi-data-sorted-small.csv.bz2", 'rt') as f:
#     content = f.readlines()

In [3]:
# rdd = sc.parallelize(content[:5000])

In [4]:
# rdd = rdd.map(lambda x: x.split(","))

In [23]:
import numpy as np
import time

In [6]:
def isfloat(value):
    try:
        float(value)
        return True
    except:
        return False

def correctRows(p):
    if(len(p) == 17):
        if(isfloat(p[4]) and isfloat(p[5]) and isfloat(p[11]) and isfloat(p[12]) and isfloat(p[16])):
            if(float(p[4]) != 0 and float(p[5]) != 0 and float(p[11]) != 0 and float(p[16])):
                return p

In [7]:
texilinesCorrect = rdd.filter(correctRows)

In [8]:
n = texilinesCorrect.count()

In [9]:
data = texilinesCorrect.map(lambda x: (np.array([1.0, float(x[4]), \
                                         float(x[5]), float(x[11]), \
                                         float(x[12]), float(x[16])])))

In [10]:
prev_w = prev_w = np.full(5, 0.1)
WtX = data.map(lambda x: (np.multiply(prev_w, x[:-1]), x))\
            .map(lambda x: (x[0].sum(), x[1]))

In [11]:
WtX.take(1)

[(12.544, array([  1.  , 120.  ,   0.44,   3.5 ,   0.5 ,   4.5 ]))]

In [12]:
errors_loss = WtX.map(lambda x: (np.subtract(x[0], x[1][5]), x[1]))\

In [20]:
import sys

In [54]:
def gradientDescent(learningRate = 1e-15, num_iteration = 100):
    prev_W= np.full(5, 0.1)
    count = 0
    # precision = 0.00000001
    # prev_stepsize = 1
    prev_loss = sys.float_info.max
    while(num_iteration > count):
        start = time.time()
        # Calculate linear combination w1 * x1 + w2 * x2.....
        # and keep original data in each row for later calculation.
        WtX = data.map(lambda x: (np.multiply(prev_W, x[:-1]), x))\
            .map(lambda x: (x[0].sum(), x[1]))

        # Calculate error which is target - prediction
        errors = WtX.map(lambda x: ((x[1][5] - x[0]), x[1]))

        # Calculate weight and loss
        dW_loss = errors.map(lambda x: (1, (np.multiply(x[1][: -1], -x[0]), x[0]**2)))\
                .reduceByKey(lambda x1, x2: (np.add(x1[0], x2[0]), x1[1] + x2[1]))\
                .collect()

        # Extract weight from list
        dW = dW_loss[0][1][0]

        # Calculate current loss 
        cur_loss = dW_loss[0][1][1]

        # Set current weight 
        cur_W = np.subtract(prev_W, learningRate*dW)



        # Bold Driver logic
        if cur_loss < prev_loss:
            learningRate += 1e-15
        else:
            learningRate -= 1e-15

        print("Iteration: ", count)
        print("W: " ,cur_W)
        print("Loss: ", cur_loss)

        count += 1

        # Calculate stepsize
        # prev_stepsize = np.mean(np.abs(np.subtract(prev_W, cur_W)))

        prev_W = cur_W

        prev_loss = cur_loss

        stop = time.time()
        duration = stop - start
        print("Duration", duration)

    return cur_W

In [55]:
W = gradientDescent()

Iteration:  0
W:  [0.09999989 0.09988687 0.09999948 0.09999808 0.09999997]
Loss:  9319805873.655169
Duration 60.69935607910156
Iteration:  1
W:  [0.09999967 0.09966093 0.09999843 0.09999425 0.0999999 ]
Loss:  9294220562.162014
Duration 26.73595118522644
Iteration:  2
W:  [0.09999934 0.09932296 0.09999686 0.09998853 0.0999998 ]
Loss:  9243226664.144451
Duration 25.149590015411377
Iteration:  3
W:  [0.0999989  0.0988742  0.09999478 0.09998093 0.09999967]
Loss:  9167210982.947403
Duration 24.413915872573853
Iteration:  4
W:  [0.09999836 0.09831635 0.0999922  0.09997148 0.09999951]
Loss:  9066765550.155823
Duration 37.21760416030884
Iteration:  5
W:  [0.09999771 0.09765157 0.09998912 0.09996022 0.09999931]
Loss:  8942679962.175713
Duration 30.24620008468628
Iteration:  6
W:  [0.09999696 0.09688241 0.09998556 0.0999472  0.09999909]
Loss:  8795931041.938976
Duration 28.496099710464478
Iteration:  7
W:  [0.09999612 0.09601189 0.09998153 0.09993246 0.09999883]
Loss:  8627670015.064383
Duration

KeyboardInterrupt: 

In [51]:
# data = texilinesCorrect.flatMap(lambda x: [(1, np.array([1.0])), (4, np.array([float(x[4])])), \
#                                         (5, np.array([float(x[5])])), (11, np.array([float(x[11])])), \
#                                         (12, np.array([float(x[12])])), (16, np.array([float(x[16])]))])

In [53]:
# def appendToCol(x1, x2):
#     col_one = np.append(x1[0], x2[0])
#     col_idx4 = np.append(x1[1], x2[1])
#     col_idx5 = np.append(x1[2], x2[2])
#     col_idx11 = np.append(x1[3], x2[3])
#     col_idx12 = np.append(x1[4], x2[4])
#     col_idx16 = np.append(x1[5], x2[5])
#     return np.array([col_one, col_idx4, col_idx5, col_idx11, col_idx12, col_idx16])

In [63]:
# prev_w = np.full(5, 0.1)
# transposed = data.reduceByKey(lambda x1, x2: np.append(x1, x2))
# errors = transposed.map(lambda x: (x[0], np.multiply(0.1, x[1])))